# **Proyek Pengembangan Machine Learning Pipeline :**
- **Nama:** Muhammad Rakha Almasah
- **Email:** muh.rakha.al@gmail.com
- **ID Dicoding:** muhrakhaal

# **Tahap 1: Pengolahan Data dalam Machine Learning Pipeline**

In [1]:
!pip install tfx tensorflow-transform --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.4 MB/s 

In [2]:
import os
from google.colab import files
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform
from tfx.proto import example_gen_pb2
import tensorflow_transform as tft

In [3]:
DATA_ROOT = '/content/data'
USERNAME_DICODING = "muhrakhaal"
PIPELINE_ROOT = f'/content/{USERNAME_DICODING}-pipeline'
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(PIPELINE_ROOT, exist_ok=True)

In [4]:
print("Silakan unggah file dataset Anda (format CSV):")
uploaded = files.upload()

Silakan unggah file dataset Anda (format CSV):


Saving cleaned_merged_stock_data.csv to cleaned_merged_stock_data.csv


In [5]:
for filename in uploaded.keys():
    uploaded_file_path = os.path.join(DATA_ROOT, filename)
    with open(uploaded_file_path, 'wb') as f:
        f.write(uploaded[filename])
    print(f"File berhasil diunggah ke {uploaded_file_path}")

DATA_FILE = uploaded_file_path

File berhasil diunggah ke /content/data/cleaned_merged_stock_data.csv


In [6]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [7]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
        ]
    )
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output_config)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [8]:
statistics_gen = StatisticsGen(examples=example_gen.outputs["examples"])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [10]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [11]:
TRANSFORM_MODULE_FILE = "nasdaq_transform.py"
with open(TRANSFORM_MODULE_FILE, 'w') as f:
    f.write('''
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "IXIC"
FEATURE_KEYS = ["AAPL", "MSFT", "AMZN", "BRK_B"]

def preprocessing_fn(inputs):
    outputs = {}
    for key in FEATURE_KEYS:
        outputs[key] = tft.scale_to_z_score(inputs[key])
    outputs[LABEL_KEY] = inputs[LABEL_KEY]

    return outputs
''')

In [12]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
context.run(transform)


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [13]:
context.show(statistics_gen.outputs["statistics"])

In [14]:
context.show(schema_gen.outputs["schema"])

Type  Presence Valency Domain
Feature name                                
'AAPL'        FLOAT  required              -
'AMZN'        FLOAT  required              -
'BRK_B'       FLOAT  required              -
'IXIC'        FLOAT  required              -
'MSFT'        FLOAT  required              -

In [15]:
context.show(example_validator.outputs["anomalies"])

# **Tahap 2: Pengembangan dan Validasi Model Machine Learning**

In [16]:
!pip install keras-tuner --quiet

In [17]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import layers
from tfx.components import Trainer, Evaluator, Tuner, Pusher
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.proto import pusher_pb2
import tensorflow_model_analysis as tfma

In [18]:
FEATURE_KEYS = ["AAPL", "MSFT", "AMZN", "BRK_B"]
LABEL_KEY = "IXIC"

In [19]:
TRAINER_MODULE_FILE = "trainer_module.py"
TUNER_MODULE_FILE = "tuner_module.py"

In [20]:
with open(TUNER_MODULE_FILE, 'w') as f:
    f.write('''
import tensorflow as tf
import keras_tuner as kt
from tensorflow_transform.tf_metadata import schema_utils
from tfx.components.trainer.fn_args_utils import FnArgs
from tensorflow_transform import TFTransformOutput

FEATURE_KEYS = ["AAPL", "MSFT", "AMZN", "BRK_B"]
LABEL_KEY = "IXIC"

def input_fn(file_pattern, tf_transform_output, num_epochs=None, batch_size=32):
    file_pattern = tf.io.gfile.glob(file_pattern)
    feature_spec = tf_transform_output.raw_feature_spec()

    def parse_fn(serialized_example):
        parsed_features = tf.io.parse_single_example(serialized_example, feature_spec)
        features = {key: parsed_features[key] for key in FEATURE_KEYS}
        label = parsed_features[LABEL_KEY]
        return features, label

    dataset = tf.data.TFRecordDataset(file_pattern, compression_type="GZIP")
    dataset = dataset.map(parse_fn)
    dataset = dataset.shuffle(buffer_size=1000).repeat(num_epochs).batch(batch_size)
    return dataset

class TunerFnResult:
    def __init__(self, tuner, fit_kwargs):
        self.tuner = tuner
        self.fit_kwargs = fit_kwargs

def tuner_fn(fn_args: FnArgs):
    tf_transform_output = TFTransformOutput(fn_args.transform_graph_path)

    def build_model(hp):
        inputs = {key: tf.keras.Input(shape=(1,), name=key) for key in FEATURE_KEYS}
        concatenated = tf.keras.layers.Concatenate()(list(inputs.values()))
        x = tf.keras.layers.Dense(
            units=hp.Int("units_1", min_value=64, max_value=256, step=64),
            activation='relu'
        )(concatenated)
        x = tf.keras.layers.Dense(
            units=hp.Int("units_2", min_value=32, max_value=128, step=32),
            activation='relu'
        )(x)
        outputs = tf.keras.layers.Dense(1)(x)
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        return model

    tuner = kt.RandomSearch(
        hypermodel=build_model,
        objective='val_mae',
        max_trials=50,
        directory=fn_args.working_dir,
        project_name="nasdaq_hyperband"
    )

    train_dataset = input_fn(fn_args.train_files, tf_transform_output)
    validation_dataset = input_fn(fn_args.eval_files, tf_transform_output)

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_dataset,
            "validation_data": validation_dataset,
            "steps_per_epoch": fn_args.train_steps,
            "validation_steps": fn_args.eval_steps
        }
    )
''')


In [21]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=20),
)
context.run(tuner)


Trial 16 Complete [00h 00m 03s]
val_mae: 5205.95654296875

Best val_mae So Far: 5045.6748046875
Total elapsed time: 00h 00m 36s
Results summary
Results in /content/muhrakhaal-pipeline/.temp/6/nasdaq_hyperband
Showing 10 best trials
Objective(name="val_mae", direction="min")

Trial 09 summary
Hyperparameters:
units_1: 192
units_2: 128
Score: 5045.6748046875

Trial 02 summary
Hyperparameters:
units_1: 128
units_2: 64
Score: 5076.2431640625

Trial 07 summary
Hyperparameters:
units_1: 192
units_2: 64
Score: 5079.44970703125

Trial 14 summary
Hyperparameters:
units_1: 128
units_2: 32
Score: 5145.517578125

Trial 15 summary
Hyperparameters:
units_1: 256
units_2: 96
Score: 5205.95654296875

Trial 13 summary
Hyperparameters:
units_1: 256
units_2: 128
Score: 5214.826171875

Trial 06 summary
Hyperparameters:
units_1: 256
units_2: 32
Score: 5239.5458984375

Trial 04 summary
Hyperparameters:
units_1: 192
units_2: 96
Score: 5255.39306640625

Trial 03 summary
Hyperparameters:
units_1: 64
units_2: 96

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [22]:
with open(TRAINER_MODULE_FILE, 'w') as f:
    f.write('''
import os
import tensorflow as tf
from tensorflow_transform import TFTransformOutput
from tfx.components.trainer.fn_args_utils import FnArgs
import json

FEATURE_KEYS = ["AAPL", "MSFT", "AMZN", "BRK_B"]
LABEL_KEY = "IXIC"

def input_fn(file_pattern, tf_transform_output, num_epochs=None, batch_size=32):
    file_pattern = tf.io.gfile.glob(file_pattern)
    feature_spec = tf_transform_output.raw_feature_spec()

    def parse_fn(serialized_example):
        parsed_features = tf.io.parse_single_example(serialized_example, feature_spec)
        features = {key: tf.cast(parsed_features[key], tf.float32) for key in FEATURE_KEYS}
        label = tf.cast(parsed_features[LABEL_KEY], tf.float32)
        return features, label

    dataset = tf.data.TFRecordDataset(file_pattern, compression_type="GZIP")
    dataset = dataset.map(parse_fn)
    dataset = dataset.shuffle(buffer_size=1000).repeat(num_epochs).batch(batch_size)
    return dataset

def build_model(hparams):
    inputs = {
        key: tf.keras.Input(shape=(1,), name=key)
        for key in FEATURE_KEYS
    }
    concatenated = tf.keras.layers.Concatenate()(list(inputs.values()))

    x = tf.keras.layers.Dense(hparams["units_1"], activation='relu')(concatenated)
    x = tf.keras.layers.Dense(hparams["units_2"], activation='relu')(x)
    outputs = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def load_best_hyperparameters(tuner_directory):
    hparams_file = os.path.join(tuner_directory, "best_hyperparameters.txt")
    if not os.path.exists(hparams_file):
        raise FileNotFoundError(f"Hyperparameter tuning file not found at {hparams_file}")

    with open(hparams_file, "r") as f:
        hparams_data = json.load(f)

    hparams = hparams_data["values"]
    return hparams

def run_fn(fn_args):
    tf_transform_output = TFTransformOutput(fn_args.transform_graph_path)

    train_dataset = input_fn(fn_args.train_files, tf_transform_output, num_epochs=fn_args.train_steps)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=1)

    tuner_uri = fn_args.custom_config["tuner_hyperparameters_path"]
    hparams = load_best_hyperparameters(tuner_uri)
    model = build_model(hparams)

    model.fit(
        train_dataset,
        validation_data=eval_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_steps=fn_args.eval_steps,
        epochs=100
    )

    os.makedirs(fn_args.serving_model_dir, exist_ok=True)

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')])
    def serve_tf_examples_fn(serialized_examples):
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        parsed_features = tf.io.parse_example(serialized_examples, raw_feature_spec)

        model_inputs = {key: parsed_features[key] for key in FEATURE_KEYS}
        return model(model_inputs)

    tf.saved_model.save(
        model,
        fn_args.serving_model_dir,
        signatures={'serving_default': serve_tf_examples_fn}
    )
''')


In [23]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=20),
    custom_config={
        "tuner_hyperparameters_path": tuner.outputs["best_hyperparameters"].get()[0].uri
    },
)
context.run(trainer)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 6045479.5000 - mae: 2271.6797 - val_loss: 47815272.0000 - val_mae: 5286.8652
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 93568192.0000 - mae: 8243.8760 - val_loss: 40695980.0000 - val_mae: 4653.6519
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 25460564.0000 - mae: 3870.6287 - val_loss: 28159642.0000 - val_mae: 3514.1211
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4088855.5000 - mae: 926.4161 - val_loss: 23614190.0000 - val_mae: 3286.3149
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39739928.0000 - mae: 5318.3149 - val_loss: 7956924.0000 - val_mae: 2084.4399
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5053026.0000 - mae: 1743.4509 - val_loss: 2232113.7500 - val_mae: 1285.5808
Epoch 7/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1398278.2500 - mae: 1013.8535 - val_loss: 1432565.7500 - val_mae: 1021.3528
Epoch 8/100
100/100 ━━━━━━━━━━━━━━━

ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [24]:
resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('latest_blessed_model_resolver')
context.run(resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [25]:
metrics_specs = [
    tfma.MetricsSpec(
        metrics=[
            tfma.MetricConfig(
                class_name='MeanSquaredError',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 500.0})
                )
            ),
            tfma.MetricConfig(
                class_name='MeanAbsoluteError',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 50.0})
                )
            )
        ]
    )
]

evaluation_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key=LABEL_KEY)],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=metrics_specs
)

evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=resolver.outputs['model'],
    eval_config=evaluation_config
)

context.run(evaluator)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [26]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=f'{PIPELINE_ROOT}/serving_model_dir'
        )
    )
)
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [27]:
context.show(evaluator.outputs['evaluation'])

# **Tahap 3: Penerapan Model Machine Learning dalam Sistem Produksi**

In [28]:
MODEL_DIR = "/content/muhrakhaal-pipeline/serving_model_dir/1734242302"
loaded_model = tf.saved_model.load(MODEL_DIR)
features = {
    "AAPL": tf.train.Feature(float_list=tf.train.FloatList(value=[176.28])),
    "MSFT": tf.train.Feature(float_list=tf.train.FloatList(value=[336.32])),
    "AMZN": tf.train.Feature(float_list=tf.train.FloatList(value=[138.12])),
    "BRK_B": tf.train.Feature(float_list=tf.train.FloatList(value=[553.50])),
    "IXIC": tf.train.Feature(float_list=tf.train.FloatList(value=[0.0]))  # IXIC sebagai placeholder
}
example = tf.train.Example(features=tf.train.Features(feature=features))
serialized_example = example.SerializeToString()
input_data = {
    'examples': tf.constant([serialized_example])
}
inference_func = loaded_model.signatures['serving_default']
predictions = inference_func(**input_data)

print("Hasil Prediksi:")
print(predictions['output_0'].numpy())


Hasil Prediksi:
[[1279065.5]]


In [30]:
!pip install flask flask-ngrok pyngrok tensorflow --quiet --ignore-installed --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.6 MB/s eta 0:00:00


In [31]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import tensorflow as tf

app = Flask(__name__)

!ngrok config add-authtoken 2qEftem6N95uNREWujMFvWdpPgB_5L4Yebd9cmvYhRxyvNyeP

MODEL_DIR = "/content/muhrakhaal-pipeline/serving_model_dir/1734242302"
loaded_model = tf.saved_model.load(MODEL_DIR)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        input_data = request.get_json()

        features = input_data.get('features', {})
        tf_features = {
            key: tf.train.Feature(float_list=tf.train.FloatList(value=value))
            for key, value in features.items()
        }
        example = tf.train.Example(features=tf.train.Features(feature=tf_features))
        serialized_example = example.SerializeToString()

        input_tensor = {'examples': tf.constant([serialized_example])}

        inference_func = loaded_model.signatures['serving_default']
        predictions = inference_func(**input_tensor)
        result = predictions['output_0'].numpy().tolist()

        return jsonify({'predictions': result})

    except Exception as e:
        return jsonify({'error': str(e)})

public_url = ngrok.connect(5000)
print(f"Ngrok URL Publik: {public_url}")

if __name__ == '__main__':
    app.run(port=5000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Ngrok URL Publik: NgrokTunnel: "https://a227-34-172-17-139.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 06:01:04] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 06:01:17] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 06:01:22] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 06:01:28] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 06:01:35] "POST /predict HTTP/1.1" 200 -


# **Download Pipeline**

In [32]:
import shutil
from google.colab import files
FOLDER_PATH = "/content/muhrakhaal-pipeline"
ZIP_FILE_PATH = "/content/muhrakhaal-pipeline.zip"
shutil.make_archive("/content/muhrakhaal-pipeline", 'zip', FOLDER_PATH)
files.download(ZIP_FILE_PATH)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>